# TensorFlow Core Learning Algorithms
- Linear Regression
- Classification
- Clustering
- Hidden Markov Models

# Linear Regression:
- One of the most basic forms of machine learning
- Predicts numberic values

In [1]:
!pip3 install -q sklearn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [2]:
%tensorflow_version 2.x # only required in notebook

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

## Load dataset

In [4]:
dftrain = pd.read_csv('./data/train.csv')
dfeval = pd.read_csv('./data/eval.csv')

y_train = dftrain.pop('survived') # pops the 'survived' column from dftrain dataframe
y_eval = dfeval.pop('survived')

# Training and testing

In [5]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']

NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []

for feature_name in CATEGORICAL_COLUMNS:
    vocabulary = dftrain[feature_name].unique() # gets all the unqiue values
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

## Training process
### Epoch:
- A single stream of entire dataset
- refers to the number of times the training model will see the entire dataset
### Batches:
- Model receives the data in batches
### Batch size:
- refers to the number of rows fed to the model at a time
### Input Function:
- defines how the data will be broken into epochs and batches before being fed to the training model

## input function:
- encodes the data into a **tf.data.Dataset** object

In [6]:
def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
    def input_function(): # Inner function, this will be returned
        ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df)) # create tf.data.Dataset object with data and its labels
        if shuffle:
            ds = ds.shuffle(1000) # randomize order of data
        ds = ds.batch(batch_size).repeat(num_epochs) # splits dataset into 32 batches and repeats process for number of epochs
        return ds # return a batch of the dataset
    return input_function # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


## Creating a model
- create linear estimaor from feature_columns created before

In [7]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpqp4s0e9m', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Training the model
- pass in the training input function

In [8]:
linear_est.train(train_input_fn) # Train
result = linear_est.evaluate(eval_input_fn) # get model metrics/stats by testing on test data

clear_output() # clears console output
print(result['accuracy'])

0.74242425


## Predictions
- making predictions

In [9]:
result = list(linear_est.predict(eval_input_fn))
clear_output()
print(dfeval.loc[0])
print(y_eval.loc[0])
print('chance of survivial: ', result[0]['probabilities'][1])
print('chance of non-survivial: ', result[0]['probabilities'][0])

sex                          male
age                          35.0
n_siblings_spouses              0
parch                           0
fare                         8.05
class                       Third
deck                      unknown
embark_town           Southampton
alone                           y
Name: 0, dtype: object
0
chance of survivial:  0.15846267
chance of non-survivial:  0.8415373


# Classification:
- Predicts class
- Seperates data points into different classes

## Load dataset

In [10]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [11]:
# https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv
# https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv

train = pd.read_csv('./data/iris_training.csv', names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv('./data/iris_test.csv', names=CSV_COLUMN_NAMES, header=0)

In [12]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [13]:
train_y = train.pop('Species')
test_y = test.pop('Species')

## Input function

In [23]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to Dataset
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle and repeat if in training mode
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

## Feature columns

In [24]:
my_features_columns = []
for key in train.keys():
    my_features_columns.append(tf.feature_column.numeric_column(key=key))
print(my_features_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


# Building the model
choices include:
- DNNSClassifier (Deep Neural Network)
- LinearClassifier

In [25]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_features_columns,
    # Two hidden layes of 30 and 10 nodes
    hidden_units=[30, 10],
    # The model must choose between 3 classes
    n_classes=3
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpyjdxgxba', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# Training:

In [29]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True), 
    steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpyjdxgxba/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpyjdxgxba/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.79971766, step = 5000
INFO:tensorflow:global_step/sec: 121.755
INFO:tensorflow:loss = 0.7966646, step = 5100 (0.823 sec)
INFO:tensorflow:global_step/sec: 210.045
INFO:tensorflow:loss = 0.79523456, step = 5200 (0.475 sec)
INFO:tensorflow:global_step/sec: 243.964
INFO:tenso

# Test trained Model's accuracy

In [30]:
eval_result = classifier.evaluate(lambda: input_fn(test, test_y, training=False ))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-07-05T12:59:17
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpyjdxgxba/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.64546s
INFO:tensorflow:Finished evaluation at 2021-07-05-12:59:17
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.53333336, average_loss = 0.79932284, global_step = 10000, loss = 0.79932284
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: /var/folders/px/lrt6ypmn0rq6y7tspwnw2ybh0000gn/T/tmpyjdxgxba/model.ckpt-10000

Test set accuracy: 0.533



# Clustering:
- Finds clusters in datasets
- Classifies each data point by finding the closest data clusters

# Hidden Markov Model:
(Probability Distributions)
- Uses probability distribution to make predict future events or states
